In [8]:
import pandas as pd

In [15]:
# Import tiny-goethe dataset text file
with open('tiny-goethe.txt', 'r', encoding='utf-8') as file:
    tiny_goethe = file.read()

In [16]:
tiny_goethe[:100]

'Prometheus. Dramatisches Fragment. 1773.\nErster Act.\nPrometheus. Merkur.\nPrometheus.\nIch will nicht,'

In [17]:
len(tiny_goethe)

6944095

In [19]:
chars = sorted(list(set(tiny_goethe)))
vocab_size = len(chars)
print(''.join(chars))
print('total chars:', vocab_size)


 !"'()*,-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz| Öàèéïòöûü“”„
total chars: 94


In [31]:
string_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_string = dict((i, c) for i, c in enumerate(chars))
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[c] for c in l])

In [30]:
decode(encode('ab hey'))

'ab hey'

In [23]:
import torch
data = torch.tensor(encode(tiny_goethe), dtype=torch.long)
print(data.shape)
data[:100]

torch.Size([6944095])


tensor([39, 71, 68, 66, 58, 73, 61, 58, 74, 72, 10,  1, 27, 71, 54, 66, 54, 73,
        62, 72, 56, 61, 58, 72,  1, 29, 71, 54, 60, 66, 58, 67, 73, 10,  1, 12,
        18, 18, 14, 10,  0, 28, 71, 72, 73, 58, 71,  1, 24, 56, 73, 10,  0, 39,
        71, 68, 66, 58, 73, 61, 58, 74, 72, 10,  1, 36, 58, 71, 64, 74, 71, 10,
         0, 39, 71, 68, 66, 58, 73, 61, 58, 74, 72, 10,  0, 32, 56, 61,  1, 76,
        62, 65, 65,  1, 67, 62, 56, 61, 73,  8])

In [24]:
n = int(len(data) * 0.8)
train_data = data[:n]
val_data = data[n:]

In [25]:
block_size = 32
train_data[:block_size+1]

tensor([39, 71, 68, 66, 58, 73, 61, 58, 74, 72, 10,  1, 27, 71, 54, 66, 54, 73,
        62, 72, 56, 61, 58, 72,  1, 29, 71, 54, 60, 66, 58, 67, 73])

In [38]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(4):
    context = x[:t+1]
    target = y[t]
    print(f"when context is {context}, target is {target}")

when context is tensor([39]), target is 71
when context is tensor([39, 71]), target is 68
when context is tensor([39, 71, 68]), target is 66
when context is tensor([39, 71, 68, 66]), target is 58


In [40]:
torch.manual_seed(1234)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

inputs
torch.Size([4, 8])
tensor([[73,  1, 66, 58, 61, 71,  1, 71],
        [72, 56, 61, 58, 67, 57, 58, 67],
        [60, 58, 60, 67, 58, 73, 58,  1],
        [72, 54, 60, 58, 67,  1, 76, 62]])
targets
torch.Size([4, 8])
tensor([[ 1, 66, 58, 61, 71,  1, 71, 74],
        [56, 61, 58, 67, 57, 58, 67,  1],
        [58, 60, 67, 58, 73, 58,  1, 62],
        [54, 60, 58, 67,  1, 76, 62, 58]])


In [52]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx) # (batch_size, block_size, vocab_size) / (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # (batch_size * block_size, vocab_size) / (BT, C)
            targets = targets.view(B*T) # (batch_size * block_size) / (BT)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(idx, None)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 94])
tensor(5.0262, grad_fn=<NllLossBackward0>)


In [53]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))


ÖÖA' ÖICkVM“voz' ?ès;éX
MkL"pHnà:òNv:]7C“0rtPàèu`YY"Kz9ö]4x.5H1.`u`Y,b1qU,O„JH“erföOöuF,Ow2G0"Q“ÖFvJ


In [54]:
optimizer = torch.optim.AdamW(m.parameters(), lr=0.001)

In [58]:
batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.1748194694519043


In [60]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, 100)[0].tolist()))


Ja gr, ben icharabr wineine un wasesih dierendond wichlfuspunges wichnen f) h n k! Zer wingeschsolle


### Mathematical trick in self-attention

Average word embeddings depending on their position in the block

In [64]:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [65]:
weights = torch.tril(torch.ones((T, T)))
weights = weights / weights.sum(dim=1, keepdim=True)
weights.shape

torch.Size([8, 8])

In [77]:
# Multiply weights with x element-wise over the last dimension using torch.einsum
x_bag_of_words = torch.einsum('ij,kjl->kil', weights, x)
torch.allclose(x_bag_of_words, weights@x)

True

In [79]:
tril = torch.tril(torch.ones((T, T)))
weights = torch.zeros((T, T))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)
x_bag_of_words = torch.einsum('ij,kjl->kil', weights, x)
weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [85]:
torch.einsum('bij,bkj -> bik', q, k).shape

torch.Size([4, 8, 8])

In [97]:
# Head
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x)
weights = torch.einsum('bij,bkj -> bik', q, k) * (head_size**-0.5)

tril = torch.tril(torch.ones((T, T)))
weights = weights.masked_fill(tril == 0, float('-inf'))
weights = F.softmax(weights, dim=-1)

v = value(x)

out = torch.einsum('kij,kjl->kil', weights, v)
out.shape

torch.Size([4, 8, 16])

In [98]:
weights[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5295, 0.4705, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2022, 0.3874, 0.4103, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2123, 0.3446, 0.2376, 0.2054, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2027, 0.2100, 0.2065, 0.1750, 0.2059, 0.0000, 0.0000, 0.0000],
        [0.1313, 0.1722, 0.1694, 0.2208, 0.1713, 0.1350, 0.0000, 0.0000],
        [0.0962, 0.2508, 0.1420, 0.1247, 0.0837, 0.1409, 0.1617, 0.0000],
        [0.1586, 0.0871, 0.1138, 0.1052, 0.1011, 0.1916, 0.1192, 0.1234]],
       grad_fn=<SelectBackward0>)